# Preparation test

In [1]:
%run "Preparation.ipynb"

# Tests

In [ ]:
df2 = pd.DataFrame({
	'customData.localplayerguid' : 1,
	'playerId' : 2,
	'C' : 3,
	'D' : '4',
	'E' : pd.Categorical(["test","train","test","train","test","train"]),
	'F' : 'aaaaaa'
	}, index=[0,1,2,3,4,5])
df2

In [ ]:
newColumns = np.unique(np.concatenate((minimalInitializationColumns, df2.columns.values)))
newColumns

In [ ]:
df3 = df2.loc[:,newColumns]
df3

In [ ]:
df4 = df3.rename(index=str, columns={'customData.localplayerguid' : 'userId', 'playerId': 'sessionId'})
df4

In [ ]:
df5 = getNormalizedRedMetricsCSV( df2 )
df5